# 🧬 Translation Efficiency Prediction with OmniGenBench

Welcome to this comprehensive tutorial where we'll explore how to predict **Translation Efficiency (TE)** from mRNA sequences using **OmniGenBench**. This guide focuses on the specific biological problem of translation efficiency and demonstrates the practical application of genomic deep learning.

> 📚 **Prerequisite**: If you are new to OmniGenBench, it is highly recommended to first study the **[Fundamental Concepts Tutorial](../00_fundamental_concepts.ipynb)**, which covers general knowledge such as language model concepts, machine learning task classification, and genomic foundation model principles.

### 1. The Biological Challenge: What is Translation Efficiency?

**Translation** is one of the most fundamental processes in molecular biology - it's the mechanism by which cells read mRNA sequences and synthesize proteins. **Translation Efficiency (TE)** quantifies how effectively this process occurs for a given mRNA molecule, directly impacting protein production levels.

Understanding and predicting TE has profound implications across multiple domains:
- **Synthetic Biology**: Designing optimized gene circuits that produce precise amounts of target proteins
- **Disease Research**: Understanding how sequence variations affect protein levels and contribute to pathogenesis
- **Biotechnology**: Engineering microorganisms for enhanced production of therapeutic proteins or industrial enzymes
- **Agriculture**: Developing crops with improved nutritional content or stress resistance

However, experimentally measuring TE across thousands of mRNA sequences is time-consuming and costly. This is where computational methods, particularly deep learning with Genomic Foundation Models, can provide transformative solutions.

### 2. The Data: Rice Translation Efficiency Dataset

To train our predictive model, we utilize a carefully curated dataset from *Oryza sativa* (rice), a model organism in plant biology.

- **What it contains**: mRNA sequences with experimentally determined translation efficiency measurements
- **What it labels**: Each sequence is classified as either "High TE" (1) or "Low TE" (0) based on ribosome profiling data
- **Our Goal**: Train a model that can accurately classify any rice mRNA sequence by its translation efficiency potential

**Dataset Structure:**

| sequence | label | 
|---------|-------|
| AUGGCCAUUGUAAUUGGCCGACUUGA... | 1 (High TE) | 
| AUGGCUACUAGCUAGCUAGCUAGC...    | 0 (Low TE) | 
| ...                                | ...  | 

Find the dataset template in **[Dataset Template](./05_advanced_dataset_creation.ipynb)** and customize it as needed for your experiments.

### 3. Quick Start: Translation Efficiency Prediction Workflow

This tutorial demonstrates the practical application of the **[Fundamental Concepts](./00_fundamental_concepts.ipynb)** to a specific biological problem. We'll use the standard 4-step OmniGenBench workflow:

```mermaid
flowchart TD
    subgraph "4-Step Workflow for TE Prediction"
        A["📥 Step 1: Data Preparation<br/>Download and process the TE dataset"] --> B["🔧 Step 2: Model Initialization<br/>Load the pre-trained OmniGenome model"]
        B --> C["🎓 Step 3: Model Training<br/>Fine-tune the model on the TE dataset"]
        C --> D["🔮 Step 4: Model Inference<br/>Use the trained model to predict TE on new sequences"]
    end

    style A fill:#e1f5fe,stroke:#333,stroke-width:2px
    style B fill:#f3e5f5,stroke:#333,stroke-width:2px
    style C fill:#e8f5e8,stroke:#333,stroke-width:2px
    style D fill:#fff3e0,stroke:#333,stroke-width:2px
```

**Translation Efficiency Prediction** is a **sequence classification** task where we predict binary labels (High TE vs Low TE) for mRNA sequences. We'll use `OmniModelForSequenceClassification` as demonstrated in the fundamental concepts tutorial.

### 4. Tutorial Structure

1. **[Data Preparation](./01_data_preparation.ipynb)**: Download and preprocess the translation efficiency dataset
2. **[Model Initialization](./02_model_initialization.ipynb)**: Load the pre-trained OmniGenome model and set it up for binary classification
3. **[Training Implementation](./03_model_training.ipynb)**: Fine-tune the model using our dataset and validate its performance
4. **[Inference: Make Predictions](./04_model_inference.ipynb)**: Use the trained model to predict translation efficiency on new mRNA sequences

Let's get started!

## 🚀 Step 1: Data Preparation

This first step is all about getting our data ready for in-silico analysis. It involves four key parts:
1.  **Environment Setup**: Installing and importing the necessary libraries.
2.  **Configuration**: Defining all our important parameters in one place.
3.  **Data Acquisition**: Downloading and preparing the raw dataset.
4.  **Data Loading**: Creating a pipeline to efficiently feed data to the model.

### 1.1: Environment Setup

First, let's install the required Python packages. `omnigenbench` is our core library, `transformers` provides the underlying model architecture, and the other packages are utilities for our workflow.

In [1]:
!pip install omnigenbench -U  # Install the latest version of omnigenbench

'pip' is not recognized as an internal or external command,
operable program or batch file.


Next, we import the libraries we just installed. This gives us the tools for data processing, deep learning, and interacting with the operating system.

A key part of this setup is determining the best available hardware for training. Our script will automatically prioritize a **CUDA-enabled GPU** if one is available, as this can accelerate training by 10-100x compared to a CPU. This makes a huge difference when working with large models and datasets.

In [ ]:
import torch

from omnigenbench import (
    ClassificationMetric,
    AccelerateTrainer,
    ModelHub,
    OmniTokenizer,
    OmniDatasetForSequenceClassification,
    OmniModelForSequenceClassification,
)


### 1.2: Global Configuration

To make our tutorial easy to modify and understand, we'll centralize all important parameters in this section. This is a best practice in software development that makes experiments more reproducible.

#### Key Parameters
-   **Dataset**: We define the local path and download URL for our dataset.
-   **Model**: We select which pre-trained OmniGenome model to use. For this tutorial, we'll start with `OmniGenome-52M` because it's fast and efficient, making it perfect for learning and prototyping.

This centralized approach allows you to easily experiment with different settings (e.g., a larger model, a different learning rate) without having to hunt through the code.

#### Note
Almost all the parameters here are standard in machine learning workflows and have a default value that works well if you don't set them. Don't worry if some of these terms are unfamiliar right now. We'll explain each one as we go along.

In [ ]:
model_name_or_path = "yangheng/OmniGenome-52M"
dataset_name = "translation_efficiency_prediction"

### 1.3: Data Acquisition

With our environment configured, it's time to download the DeepSEA dataset. The function below automates this process by:
1.  Checking if the data already exists.
2.  Downloading the dataset from the specified URL if needed.
3.  Extracting the files.
4.  Cleaning up the temporary zip file.

This ensures we have the `train.jsonl`, `valid.jsonl`, and `test.jsonl` files ready for the next stage. These files represent the standard splits for training, validating, and testing our model.

In [ ]:
# Model and Tokenizer

# We define the label mapping in the training
label2id = {"0": 0, "1": 1}  # 0: Low TE, 1: High TE

# Initialize tokenizer
tokenizer = OmniTokenizer.from_pretrained(model_name_or_path)
print(f"✅ Tokenizer loaded: {model_name_or_path}")

# Load datasets using the enhanced OmniDataset framework
print("🏗️ Loading datasets with automatic download...")
datasets = OmniDatasetForSequenceClassification.from_huggingface(
    dataset_name="translation_efficiency_prediction",
    tokenizer=tokenizer,
    max_length=512,
    label2id=label2id,
)
print(f"📊 Datasets loaded: {list(datasets.keys())}")
for split, dataset in datasets.items():
    print(f"  - {split}: {len(dataset)} samples")

### 1.4: Dataset Loading with OmniGenBench

With OmniGenBench, data loading is significantly simplified! The framework automatically handles:

#### Automatic Data Processing
The `OmniDatasetForSequenceClassification` class automatically:
1. **Downloads and processes** the dataset from our curated collection
2. **Handles sequence preprocessing** including truncation, padding, and tokenization
3. **Manages binary classification formatting** for translation efficiency prediction
4. **Creates train/validation/test splits** ready for training

This streamlined approach eliminates the need for custom dataset classes while maintaining full flexibility and performance.

In [ ]:
print("📝 Data loading completed! Using  OmniDataset framework.")
print(f"📊 Loaded datasets: {list(datasets.keys())}")
for split, dataset in datasets.items():
    print(f"  - {split}: {len(dataset)} samples")

## 🚀 Step 2: Model Initialization

With our data pipeline in place, it's time to set up the model. This is where the power of Genomic Foundation Models (GFMs) comes into play. Instead of building a model from scratch, we will load the pre-trained **OmniGenome** model and adapt it for our specific task.

This process involves three key components:
1.  **The Tokenizer**: This is the same tokenizer we used in the data preparation step. It's responsible for converting DNA sequences into a numerical format the model can process. It's crucial that we use the *exact same* tokenizer that the model was pre-trained with.
2.  **The Base Model**: This is the core OmniGenome model, which has already learned the fundamental patterns of genomic sequences from its extensive pre-training. We load it directly from the Hugging Face Hub.
3.  **The Classification Head**: The base model is great at understanding sequences, but it doesn't know anything about our specific TFB prediction task. To fix this, we add a "classification head" on top of the base model. This is a simple neural network layer that takes the sequence representation from the base model and maps it to our desired output, in this case, a prediction for each of the TF labels.


The `OmniModelForMultiLabelSequenceClassification` class from our library handles this process for us, seamlessly combining the base model and the new classification head.
In adddition, most of the models and datasets have been integrated into the OmniGenBench package, making it easy to load them with just a few lines of code. Please refer to the sub-tutorials of [data preparation](./02_data_preparation.ipynb) and [model initialization](./03_model_initialization.ipynb) for more details.

In [ ]:
# === Model Initialization ===
# We support all genomic foundation models from Hugging Face Hub.

model = OmniModelForSequenceClassification(
    model_name_or_path,
    tokenizer,
    num_labels=len(list(label2id.keys())),  # Binary classification: Low TE vs High TE
)


## 🚀 Step 3: Model Training

This is the most exciting part! With our data and model ready, we can now begin the **fine-tuning** process. During training, the model will learn to associate specific patterns in the mRNA sequences with high or low translation efficiency.

The `AccelerateTrainer` from `omnigenbench` wraps all this logic into a simple interface, allowing us to launch the training process with just a few lines of code.

In [ ]:
metric_functions = [ClassificationMetric().f1_score]

trainer = AccelerateTrainer(
    model=model,
    train_dataset=datasets["train"],
    eval_dataset=datasets["valid"],
    test_dataset=datasets["test"],
    compute_metrics=metric_functions,
)
print("🎓 Starting training...")

metrics = trainer.train()
trainer.save_model("ogb_te_finetuned")

print('Metrics:', metrics)

## 🔮 Step 4: Model Inference and Interpretation

Now that we have a trained model, let's use it for its intended purpose: predicting translation efficiency on new mRNA sequences. This process is called **inference**.

### The Inference Pipeline

Our inference pipeline consists of a few key steps:
1. **Load the Model**: We load the best-performing model that was saved during training.
2. **Process the Input**: We take new mRNA sequences and apply the same preprocessing steps we used for our training data (truncating/padding and tokenizing).
3. **Run Prediction**: We feed the processed sequence to the model and get its predictions. We use `torch.no_grad()` to disable gradient calculations, which makes inference faster and uses less memory.
4. **Interpret the Results**: The model's raw output is a probability score. We'll interpret these to make them more understandable, identifying whether sequences have high or low translation efficiency and with what level of confidence.

To demonstrate, we'll test our model on a few sample sequences and print out a user-friendly summary of the results. This shows how the model can be used in a real-world application to analyze sequences of interest.

In [ ]:

inference_model = ModelHub.load("yangheng/ogb_te_finetuned")

sample_sequences = {
    "Optimized sequence": "AAACCAACAAAATGCAGTAGAAGTACTCTCGAGCTATAGTCGCGACGTGCTGCCCCGCAGGAGTACAGTAGTAGTACAACGTAAGCGGGAGCAACAGACTCCCCCCCTGCAACCCACTGTGCCTGTGCCCTCGACGCGTCTCCGTCGCTTTGGCAAATGTCACGTACATATTACCGTCTCAGGCTCTCAGCCATGCTCCCTACCACCCCTGCAGCGAAGCAAAAGCCACGCACGCGGCGCCTGACATGTAACAGGACTAGACCATCTTGTTCATTTCCCGCACCCCCTCCTCTCCTCTTCCTCCATCTGCCTCTTTAAAACAGTAAAAATAACCGTGCATCCCCTGGGCAAAATCTCTCCCATACATACACTACAGCGGCGAACCTTTCCTTATTCTCGCAACGCCTCGGTAACGGGCAGCGCCTGCTCCGCGCCGCGGTTGCGAGTTCGGGAAGGCGGCCGGAGTCGCGGGGAGGAGAGGGAGGATTCGATCGGCCAGA",
    "Suboptimal sequence": "TGGAGATGGGCAGATGGCACACAAAACATGAATAGAAAACCCAAAAGGAAGGATGAAAAAAACACACACACACACACACACAAAACACAGAGAGAGAGAGAGAGAGAGCGAGAAAAGAAAAGAAAAAACCAATTCTTTTGGTCTCTTCCCTCTCCGTTTGTCGTGTCGAAGCCTTTGCCCCCACCACCTCCTCCTCTCCTCTCCCTTCCTCCCCTCCTCCCCATCTCGCTCTCCTCCCTCCTCTCTCCTCTCCTCGTCTCCTCTTCCTCTCCATTCCATTGGCCATTCCATTCCATTCCACCCCCCATGAAACCCCAAACCCTCGTCGGCCTCGCCGCGCTCGCGTAGCGCACCCGCCCTTCTCCTCTCGCCGGTGGTCCGCCGCCAGCCTCCCCCCACCCGATCCCGCCGCCCCCCCCGCCTTCACCCCGCCCACGCGGACGCATCCGATCCCGCCGCATCGCCGCGCGGGGGGGGGGGGGGGGGGGGGGGGGAGGGCACG",
    "Random sequence": "AUGC" * (128 // 4),
}
with torch.no_grad():

    for seq_name, sequence in sample_sequences.items():
        outputs = inference_model.inference(sequence)
        print("✅ Prediction completed!")

        # —— Result Interpretation ——
        prediction = outputs.get('predictions', [0])[0]
        probability = outputs.get('probabilities', [0.5])[0] if 'probabilities' in outputs else 0.5

        te_class = "High TE" if prediction == 1 else "Low TE"
        confidence = probability if prediction == 1 else (1 - probability)

        print(f"📊 Analysis for {seq_name}:")
        print(f"  🎯 Prediction: {te_class}")
        print(f"  📈 Confidence: {confidence:.3f}")
        print(f"  📏 Sequence length: {len(sequence)} nucleotides")

        if confidence > 0.8:
            emoji = "🟢 High confidence"
        elif confidence > 0.6:
            emoji = "🟡 Moderate confidence"
        else:
            emoji = "🔴 Low confidence"
        print(f"  {emoji} Model confidence level for this prediction\n")
        

## 🎉 Tutorial Summary and Next Steps

Congratulations! You have successfully completed this comprehensive tutorial on translation efficiency prediction with OmniGenBench.

### What You've Learned

You've walked through a complete, end-to-end application of genomic deep learning, demonstrating how the concepts from the **[Fundamental Concepts Tutorial](../00_fundamental_concepts.ipynb)** apply to a real biological problem. Specifically, you have:

1. **Applied Task Formulation**: Successfully framed translation efficiency prediction as a sequence classification problem
2. **Mastered the 4-Step Workflow**:
   - **Step 1: Data Preparation**: Acquired, processed, and loaded the rice translation efficiency dataset
   - **Step 2: Model Initialization**: Set up OmniModelForSequenceClassification for binary classification
   - **Step 3: Model Training**: Fine-tuned the model using best practices and appropriate evaluation metrics
   - **Step 4: Model Inference**: Generated predictions on new mRNA sequences and interpreted results
3. **Understood Practical Application**: Gained hands-on experience with a biologically relevant prediction task

### 🚀 Next Steps

Now that you've mastered translation efficiency prediction, you can:

#### 🧬 **Explore Other Sequence Classification Tasks**
- **Promoter Recognition**: Identify regulatory sequences
- **Subcellular Localization**: Predict protein cellular destinations  
- **Functional Annotation**: Classify protein or RNA functions

#### 📊 **Try Different Task Types**
- **Sequence Regression**: Gene expression level prediction
- **Token Classification**: Binding site identification
- **Multi-label Classification**: Multi-functional sequence prediction

#### 🔬 **Advanced Techniques**
- **Custom Dataset Creation**: Use the [Advanced Dataset Creation Tutorial](./05_advanced_dataset_creation.ipynb)
- **Model Comparison**: Benchmark different foundation models
- **Hyperparameter Optimization**: Fine-tune model performance
- **Biological Validation**: Compare predictions with experimental data

### 📚 Resources

- **[Fundamental Concepts Tutorial](../00_fundamental_concepts.ipynb)**: Review core concepts anytime
- **[OmniGenBench Documentation](https://omnigenbench.readthedocs.io/)**: Complete API reference
- **[GitHub Repository](https://github.com/yangheng95/OmniGenBench)**: Source code and community discussions

Thank you for following along. We hope this tutorial has provided you with the knowledge and confidence to apply deep learning to your own genomics research. Happy coding!